In [1]:
import os
%pwd

'c:\\Users\\dengz\\Desktop\\Projects\\cnn_classifier\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\dengz\\Desktop\\Projects\\cnn_classifier'

In [3]:
# import dagshub
# dagshub.init(repo_owner='AndyDengFKu', repo_name='cnn_classifier', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    params_num_classes: int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
            eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.pt"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_num_classes=self.params.CLASSES
        )
            return eval_config



In [7]:
import torch
import dagshub
import mlflow
import mlflow.pytorch
import torch.nn.functional as F
import torchvision.models as models

from torch import nn
from urllib.parse import urlparse
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [8]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        # 1. 构建验证集 DataLoader
        normalize = transforms.Normalize(mean=[0.485,0.456,0.406],
                                         std=[0.229,0.224,0.225])
        valid_transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:2]),
            transforms.ToTensor(),
            normalize
        ])
        full_ds = datasets.ImageFolder(self.config.training_data,
                                       transform=valid_transform)
        val_size = int(0.2 * len(full_ds))
        _, valid_ds = random_split(full_ds,
                                   [len(full_ds)-val_size, val_size])
        self.valid_loader = DataLoader(valid_ds,
                                       batch_size=self.config.params_batch_size,
                                       shuffle=False,
                                       num_workers=4)

    @staticmethod
    def load_model(path: Path, num_classes: int, device):
        model = models.resnet18(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        state = torch.load(path, map_location=device)
        model.load_state_dict(state)
        return model.to(device)

    def evaluate(self):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.load_model(self.config.path_of_model,
                                     self.config.params_num_classes,
                                     device)
        self.model.eval()
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for X, y in self.valid_loader:
                X, y = X.to(device), y.to(device)
                logits = self.model(X)
                loss = F.cross_entropy(logits, y, reduction='sum')
                total_loss += loss.item()
                correct += (logits.argmax(1) == y).sum().item()
                total += y.size(0)
        self.val_loss = total_loss / total
        self.val_acc = correct / total

    def save_score(self):
        scores = {"loss": self.val_loss, "accuracy": self.val_acc}
        save_json(path=Path("scores.json"), data=scores)

    def evaluation(self):
  
        # 1. dagshub + mlflow init
        dagshub.init(
            repo_owner='AndyDengFKu',
            repo_name='cnn_classifier',
            mlflow=True
        )
        # # 2. 设置 MLflow tracking server URI
        # mlflow.set_tracking_uri(self.config.mlflow_uri)

        # 获取 Tracking URI 的 scheme，用于判断 file store vs. 远程
        tracking_scheme = urlparse(mlflow.get_tracking_uri()).scheme

        # 3. 开启一个 MLflow run
        with mlflow.start_run():
            # 4. 评估 & 保存分数
            self.evaluate()
            self.save_score()

            # 5. 一次性记录所有超参数
            mlflow.log_params(self.config.all_params)

            # 6. 记录关键指标
            mlflow.log_metric('val_loss', self.val_loss)
            mlflow.log_metric('val_accuracy', self.val_acc)

            # 7. 上传 scores.json（可选）
            mlflow.log_artifact("scores.json")

            # 8. 保存模型 artifact
            if tracking_scheme != "file":
                # 远程 Registry：带注册名
                mlflow.pytorch.log_model(
                    self.model,
                    artifact_path="models",
                    registered_model_name="ResNet18Model"
                )
            else:
                # 本地 file store：只当做 artifact 保存
                mlflow.pytorch.log_model(
                    self.model,
                    artifact_path="models"
                )


In [9]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
except Exception as e:
    raise e

[2025-04-26 21:30:25,284: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-26 21:30:25,287: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-26 21:30:25,288: INFO: common: created directory at: artifacts]
[2025-04-26 21:30:25,800: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as AndyDengFKu

[2025-04-26 21:30:25,805: INFO: helpers: Accessing as AndyDengFKu]
[2025-04-26 21:30:26,330: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/AndyDengFKu/cnn_classifier "HTTP/1.1 200 OK"]
[2025-04-26 21:30:26,820: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "AndyDengFKu/cnn_classifier"

[2025-04-26 21:30:26,825: INFO: helpers: Initialized MLflow to track repo "AndyDengFKu/cnn_classifier"]


...

[2025-04-26 21:30:26,826: INFO: helpers: Repository AndyDengFKu/cnn_classifier initialized!]


c:\Users\dengz\anaconda3\envs\cnn_classifier\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dengz\anaconda3\envs\cnn_classifier\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\dengz\AppData\Local\Temp\ipykernel_4572\4037134188.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default

[2025-04-26 21:30:31,845: INFO: common: json file saved at: scores.json]


2025/04/26 21:30:33 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/26 21:30:41 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/26 21:30:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ResNet1

🏃 View run kindly-slug-156 at: https://dagshub.com/AndyDengFKu/cnn_classifier.mlflow/#/experiments/0/runs/ce8c1aa2d3774a109490e16a36c11bb2
🧪 View experiment at: https://dagshub.com/AndyDengFKu/cnn_classifier.mlflow/#/experiments/0
